In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


## 1. Data Generation (simulating air pollution data)


In [3]:
def generate_air_pollution_data(n_samples=1000):
    np.random.seed(42)
    
    # Features: industrial_emissions, vehicle_traffic, temperature, wind_speed, precipitation
    industrial_emissions = np.random.normal(50, 20, n_samples)
    vehicle_traffic = np.random.normal(1000, 300, n_samples)
    temperature = np.random.normal(20, 5, n_samples)
    wind_speed = np.random.normal(10, 3, n_samples)
    precipitation = np.random.normal(50, 20, n_samples)
    
    # Target: air_quality_index (AQI) (simplified model)
    aqi = (
        0.3 * industrial_emissions +
        0.3 * vehicle_traffic +
        0.1 * temperature +
        -0.2 * wind_speed +
        -0.1 * precipitation +
        np.random.normal(0, 10, n_samples)
    )
    aqi = np.clip(aqi, 0, 500)  # AQI typically ranges from 0 to 500
    
    data = pd.DataFrame({
        'industrial_emissions': industrial_emissions,
        'vehicle_traffic': vehicle_traffic,
        'temperature': temperature,
        'wind_speed': wind_speed,
        'precipitation': precipitation,
        'aqi': aqi
    })
    
    return data

## 2. Data Preprocessing


In [4]:
def preprocess_data(data):
    X = data[['industrial_emissions', 'vehicle_traffic', 'temperature', 'wind_speed', 'precipitation']]
    y = data['aqi']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test, scaler

## 3. Model Creation


In [5]:
def create_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

## 4. Model Training


In [6]:
def train_model(model, X_train, y_train):
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
    return history

## 5. AQI Category Function


In [7]:
def get_aqi_category(aqi):
    if aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Moderate"
    elif aqi <= 150:
        return "Unhealthy for Sensitive Groups"
    elif aqi <= 200:
        return "Unhealthy"
    elif aqi <= 300:
        return "Very Unhealthy"
    else:
        return "Hazardous"

## 6. Question Generation


In [8]:
def generate_question(model, scaler, question_type='multiple_choice'):
    # Generate random input
    input_data = np.random.rand(1, 5)
    scaled_input = scaler.transform(input_data)
    
    # Predict AQI
    predicted_aqi = model.predict(scaled_input)[0][0]
    aqi_category = get_aqi_category(predicted_aqi)
    
    # Create question based on the prediction
    if question_type == 'multiple_choice':
        question = "Given the following conditions:\n"
        question += f"Industrial Emissions: {input_data[0][0]*100:.2f} µg/m³\n"
        question += f"Vehicle Traffic: {input_data[0][1]*2000:.0f} vehicles/hour\n"
        question += f"Temperature: {input_data[0][2]*40:.2f}°C\n"
        question += f"Wind Speed: {input_data[0][3]*20:.2f} km/h\n"
        question += f"Precipitation: {input_data[0][4]*100:.2f} mm\n\n"
        question += "What is the most likely Air Quality Index (AQI) category?\n"
        
        options = [
            "A) Good",
            "B) Moderate",
            "C) Unhealthy for Sensitive Groups",
            "D) Unhealthy",
            "E) Very Unhealthy",
            "F) Hazardous"
        ]
        
        correct_answer = options[[opt.split(') ')[1] for opt in options].index(aqi_category)].split(') ')[0]
        
        return question, options, correct_answer
    
    elif question_type == 'true_false':
        statement = "Given the environmental conditions, "
        if predicted_aqi > 150:
            statement += "the air quality is likely to be unhealthy or worse."
            correct_answer = True
        else:
            statement += "the air quality is likely to be moderate or good."
            correct_answer = False
        
        return statement, correct_answer

## 7. Main Execution


In [9]:
if __name__ == "__main__":
    # Generate and preprocess data
    data = generate_air_pollution_data()
    X_train, X_test, y_train, y_test, scaler = preprocess_data(data)
    
    # Create and train model
    model = create_model((X_train.shape[1],))
    history = train_model(model, X_train, y_train)
    
    # Evaluate model
    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test MAE: {test_mae:.4f}")
    
    # Generate questions
    print("\nMultiple Choice Question:")
    mc_question, mc_options, mc_answer = generate_question(model, scaler, 'multiple_choice')
    print(mc_question)
    for option in mc_options:
        print(option)
    print(f"Correct Answer: {mc_answer}")
    
    print("\nTrue/False Question:")
    tf_statement, tf_answer = generate_question(model, scaler, 'true_false')
    print(tf_statement)
    print(f"Correct Answer: {tf_answer}")

/Users/rajesh/Desktop/Code/NASA/5-AP/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test MAE: 17.5767

Multiple Choice Question:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Given the following conditions:
Industrial Emissions: 33.65 µg/m³
Vehicle Traffic: 422 vehicles/hour
Temperature: 12.53°C
Wind Speed: 3.58 km/h
Precipitation: 90.19 mm

What is the most likely Air Quality Index (AQI) category?

A) Good
B) Moderate
C) Unhealthy for Sensitive Groups
D) Unhealthy
E) Very Unhealthy
F) Hazardous
Correct Answer: E

True/False Question:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Given the environmental conditions, the air quality is likely to be unhealthy or worse.
Correct Answer: True


/Users/rajesh/Desktop/Code/NASA/5-AP/.venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rajesh/Desktop/Code/NASA/5-AP/.venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## 8. Interactive Quiz Function


In [10]:
def take_quiz(model, scaler, num_questions=5):
    score = 0
    for i in range(num_questions):
        print(f"\nQuestion {i+1}:")
        if np.random.rand() > 0.5:
            # Multiple Choice Question
            question, options, correct_answer = generate_question(model, scaler, 'multiple_choice')
            print(question)
            for option in options:
                print(option)
            user_answer = input("Your answer (A/B/C/D/E/F): ").upper()
            if user_answer == correct_answer:
                print("Correct!")
                score += 1
            else:
                print(f"Incorrect. The correct answer is {correct_answer}.")
        else:
            # True/False Question
            statement, correct_answer = generate_question(model, scaler, 'true_false')
            print(statement)
            user_answer = input("True or False? (T/F): ").upper() == 'T'
            if user_answer == correct_answer:
                print("Correct!")
                score += 1
            else:
                print(f"Incorrect. The correct answer is {correct_answer}.")
    
    print(f"\nQuiz completed! Your score: {score}/{num_questions}")

## 9. Run the quiz


In [ ]:
if __name__ == "__main__":
    print("\nWelcome to the Air Pollution Impact Quiz!")
    take_quiz(model, scaler)

In [1]:
import joblib
from sklearn.ensemble import RandomForestClassifier

# Create a scikit-learn model
model = RandomForestClassifier()

# Train the model on your data

# Save the model to a file
joblib.dump(model, 'AP.pkl')

['AP.pkl']